
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [1]:
from threading import Thread, Lock, Event
import time
# Defino una clase Barberia en la que meteré a mis clientes y al barbero
class Barberia:
        # Creo un cerrojo que me permitirá utilizarlo para la excusion mutua
        cerrojo = Lock()
        clientes_esperando = []

        def __init__(self, barbero):
                # Meto al barbero en la clase de la barbería.
                self.barbero = barbero

        # Defino un atributo que crea el thread que activa al barbero trabajador.
        def abrir_barberia(self):
                print( 'Se abre la barberia')
                thread_activo = Thread(target = self.barbero_empieza_a_trabajar)
                thread_activo.start()
                
        # Creo una actividad para el barbero, que se parasá dentro de un thread
        def barbero_empieza_a_trabajar(self):
                while True:
                        # Cierro el cerrojo, exclusion mutua. Si hay algún cliente, despierto al barbero. 
                        self.cerrojo.acquire()
                        if len(self.clientes_esperando) > 0 :
                                # Llamo al cliente y lo quito de la lista
                                cliente = self.clientes_esperando[0]
                                del self.clientes_esperando[0]
                                # Libero el cerrojo
                                self.cerrojo.release()
                                # Empiezo a cortar el pelo al cliente
                                self.barbero.corta_pelo(cliente)
                        else:
                                self.cerrojo.release()
                                print('El barbero se echa a dormir')
                                barbero.duerme()
                                # Este print solo saldrá si el barbero se despierta, es decir
                                # si el evento sale del estado wait. Esto solo pasrá si entra un cliente
                                # porque corta_pelo lo desactiva
                                print ('El barbero se despierta')

        def entrar_a_barberia(self, customer):
                # Cierro el cerrojo, exclusion mutua
                self.cerrojo.acquire()
                # Permito al cliente entrar y buscar un asiento
                print ('{0} entra y busca un asiento'.format(customer.name))
                # Si la barberia está llena, se marcha
                if len(self.clientes_esperando) == 3:
                        print ('Todas las sillas están cogidas, {0} se marcha.'.format(customer.name))
                        # Abro el cerrojo 
                        self.cerrojo.release()
                else:
                        print ('{0} se sienta en una silla'.format(customer.name))
                        # Meto en la lista de clientes esperando al nuevo cliente
                        self.clientes_esperando.append(cliente_llegando)
                        self.cerrojo.release()
                        # Levanta al barbero, que pone la flag de dormido a True y entonces SÍ se activa el pring anterior.
                        barbero.despierta()
                        
# Creamos una clase para el barbero y definimos sus actividades
class Barbero:
        def __init__(self, name):
                self.name = name

        # Creamos un evento interno con el que controlaremos el estado del barbero
        evento_barbero = Event()

        # Deja al barbero dormido hasta que alguien lo despierte. 
        def duerme(self):
                self.evento_barbero.wait()

        def despierta(self):
                # Devuelvo el evento del barbero a true
                self.evento_barbero.set()

        def corta_pelo(self, customer):
                # Quito los estados del barbero
                self.evento_barbero.clear()
                print ('A %s le están cortando el pelo' %customer.name)
                time.sleep(10)
                print ('{0} ha acabado de cortarse el pelo '.format(customer.name))
 
                    
# Creamos una clase para el cliente y le damos un nombre al cliente.               
class Cliente:
        def __init__(self, name):
                self.name = name
               
                
                
if __name__ == '__main__':
        # Creo una lista con clientes que irán entrando en la tienda. 
        clientes = []
        clientes.append(Cliente('Adan'))
        clientes.append(Cliente('Carlos'))
        clientes.append(Cliente('Neus'))
        clientes.append(Cliente('Maria'))
        clientes.append(Cliente('Pedro'))
        clientes.append(Cliente('Paco'))
        clientes.append(Cliente('Loles'))

        # Inicializo al barbero
        barbero = Barbero('Barbero')
        
        # Inicializo la barberia
        barberShop = Barberia(barbero)
        barberShop.abrir_barberia()

        # Mientras la lisa tenga clientes, irán intentando ir a cortarse el pelo.
        while len(clientes) > 0:
                cliente_llegando = clientes.pop()
                barberShop.entrar_a_barberia(cliente_llegando)
                # Que esperen para ir llegando
                time.sleep(3)
                

Se abre la barberia
El barbero se echa a dormir
Loles entra y busca un asiento
Loles se sienta en una silla
El barbero se despierta
A Loles le están cortando el pelo
Paco entra y busca un asiento
Paco se sienta en una silla
Pedro entra y busca un asiento
Pedro se sienta en una silla
Maria entra y busca un asiento
Maria se sienta en una silla
Loles ha acabado de cortarse el pelo 
A Paco le están cortando el pelo
Neus entra y busca un asiento
Neus se sienta en una silla
Carlos entra y busca un asiento
Todas las sillas están cogidas, Carlos se marcha.
Adan entra y busca un asiento
Todas las sillas están cogidas, Adan se marcha.
Paco ha acabado de cortarse el pelo 
A Pedro le están cortando el pelo
Pedro ha acabado de cortarse el pelo 
A Maria le están cortando el pelo
Maria ha acabado de cortarse el pelo 
A Neus le están cortando el pelo
Neus ha acabado de cortarse el pelo 
El barbero se echa a dormir


# CODIGO QUE NO FUNCIONA
Adjunto también el código que he hecho siguiendo el esquema de clase. Mi problema con este código es que no consigo "ordenar" los prints para que tengan un sentido, pero funcionar funciona perfectamente. Probablemente sea por los times o por la exclusión mutua.

In [7]:
import threading
import time
import sys
import random
class Barberia():
    def __init__(self):
        
        ############################################################
        ## Control de los clientes
        ############################################################
        # Solo puedo tener 3 clientes en la sala de espera. Lo hago con un semáforo.
        # Ojo, estar en la sala != de estar en la silla.
        self.entrarEnSala = threading.Semaphore(3)
        #El numero de clientes en la sala se incializa a cero. 
        self.clientesEnSala = 0
        # Garantiza exclusion para modificacion de variables de control de sala o acceso a secciones.
        # Asi evito que cambie el numero de clientes sin poder controlarlo.
        self.exclusionMutuaSala=threading.Semaphore()
        ############################################################
        ## Control del barbero
        ############################################################
        # El sillon será un cerrojo, así si te sientas no te puedes levantar. No queremos que el cliente se vaya a medio corte de pelo.
        self.sillon = threading.Condition()
        
        # Podemos tener al barbero ocupado o dormido. De base, el barbero estará libre (ocupado = false)  y dormido
        self.barbero_ocupado=False
        #puede poner al barbero a dormir y lo uso como condicion para el wait
        self.barbero_dormido=threading.Condition()
       
       
    def actividadBarbero(self):
        # Bucle infinito
        while True:
            # Exclusion mutua para que el numero de clientes no varie
            self.exclusionMutuaSala.acquire() 
            if self.clientesEnSala > 0:
                self.exclusionMutuaSala.release()
                print("El barbero da paso a un cliente\n")
                ###  INTRODUCE EL CÓDIGO PARA CAMBIAR EL ESTADO DEL BARBERO ###
                print("El barbero corta a un cliente\n")
                with self.sillon: #Bloqueamos la condición sillon
                    self.barberoOcupado=True #Pasamos el booleanoa a ocupado
                    print('El barbero está cortando el pelo \n')  
                time.sleep(3) #Simulamos el corte con un tiempo de 3 unidades
                with self.sillon: #Bloqueamos la condición sillon
                    self.barberoOcupado=False #Pasamos el booleano a libre
                    print('El barbero termina de cortar el pelo \n')
                    self.sillon.notify() #Notificamos a los clientes que esten esperando
            else:
                self.exclusionMutuaSala.release()
                self.barbero_ocupado=False
                with self.barbero_dormido:  
                    print('El barbero esta dormido \n')
                    self.barbero_dormido.wait()

            

    def actividadCliente(self): 
            print("El cliente "+threading.currentThread().name+" llega a la puerta de la barbería \n")
            #Si no hay 3 esperando, puedo entrar. 
            if self.clientesEnSala < 3: 
                print("El cliente "+threading.currentThread().getName()+" entra en la sala\n")
                # Aviso al semaforo de que hay una persoan mas en la sala y la meto en la lista
                self.entrarEnSala.acquire()
                self.clientesEnSala +=1 
            else:
                #Aqui podria poner un bucle y cola para que espern en la puerta o reintentar la entrada.
                print("El cliente "+threading.currentThread().getName()+" se marcha por estar lleno \n")
                # Si el cliente se va, cierro su hilo
                sys.exit()
            #Para que no haya varias personas consultando al barbero
            self.exclusionMutuaSala.acquire() 
            with self.sillon:
                # Si el barbero está ocupado, quiere decir que también está despierto y trabajando. Asi que se pone a esperar al sillon
                if self.barbero_ocupado:
                    print ("El cliente "+threading.currentThread().name+" espera\n")
                    self.exclusionMutuaSala.release()
                    self.sillon.wait() # A la espera del sillon 
                    time.sleep(3)
                    print ("Esta en el sillon  "+threading.currentThread().name+"\n")
                    self.barbero_ocupado = True
                else:
                    # Sabemos que si el barbero no esta ocupado, esta dormido, asi que lo despierta y se sienta en el sillon
                    with self.barbero_dormido:  
                        print("El cliente "+threading.currentThread().getName()+" despierta al barbero\n")
                        self.barbero_dormido.notify()
                    # Quito al cliente de la sala porque esta en el sillon
                    print ("Esta en el sillon  "+threading.currentThread().name+"\n")
                    time.sleep(3)
                    self.barbero_ocupado = True
                    self.exclusionMutuaSala.release() 
           
            print ("El cliente "+threading.currentThread().name+" abandona la barbería\n")
            self.clientesEnSala -=1 
            self.entrarEnSala.release()
            

if __name__=="__main__":
        barberia= Barberia()                   
        barbero =  threading.Thread(target = barberia.actividadBarbero)
        barbero.start()
        lista_clientes = ['Adan', 'Carlos', 'Maria',
                          'Pedro', 'Paco', 'Loles']
        for i in lista_clientes:
            #Simulamos tiempo entre clientes
            time.sleep(random.randint(0,2)) 
            cliente=threading.Thread(target=barberia.actividadCliente, name=str(i))
            cliente.start() 
        #Hasta que no pare el ultimo cliente no paro 
        cliente.join() 

El barbero esta dormido 

El cliente Adan llega a la puerta de la barbería 

El cliente Adan entra en la sala

El cliente Adan despierta al barbero

Esta en el sillon  Adan

El cliente Carlos llega a la puerta de la barbería 

El cliente Carlos entra en la sala

El cliente Maria llega a la puerta de la barbería 

El cliente Maria entra en la sala

El cliente Pedro llega a la puerta de la barbería 

El cliente Pedro se marcha por estar lleno 

El cliente Paco llega a la puerta de la barbería 

El cliente Paco se marcha por estar lleno 

El cliente Adan abandona la barbería
El barbero da paso a un cliente
El cliente Carlos espera



El barbero corta a un cliente

El barbero está cortando el pelo 

El cliente Maria espera

El cliente Loles llega a la puerta de la barbería 

El cliente Loles entra en la sala

El cliente Loles espera

El barbero termina de cortar el pelo 

El barbero da paso a un cliente

El barbero corta a un cliente

El barbero está cortando el pelo 

Esta en el sillon  C